<h3>Unsupervised Learning Trading Strategy</h3>

- Download/Load SP500 stocks prices data.
- Calculate different features and indicators on each stock.
- Aggregate on monthly level and filter top 150 most liquid stocks.
- Calculate Monthly Returns for different time-horizons.
- Download Fama-French Factors and Calculate Rolling Factor Betas.
- For each month fit a K-Means Clustering Algorithm to group similar assets based on their features.
- For each month select assets based on the cluster and form a portfolio based on Efficient Frontier max sharpe ratio optimization.
- Visualize Portfolio returns and compare to SP500 returns. 

<h3>All Packages</h3>
- pandas
- numpy
- matplotlib
- statsmodels
- pandas_datareader
- datetime
- yfinance
- sklearn
- PyPortfolioOpt

<h3>Download / Load SP500 stocks prices data.</h3>

In [22]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

#Not Survivorship Bias free
symbols_list = sp500['Symbol'].unique().tolist()

# end_date = '2023-09-27'

# start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

# df = yf.download(tickers=symbols_list,
#                start=start_date,
#                end=end_date).stack()

# df.index.names = ['date', 'ticker']
# df

# df.columns = df.columns.str.lower()
# df



Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.425238   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.651133   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.004166   52.790001   54.189999   51.880001   53.099998   
           ABT      33.302025   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.736298   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211006  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.131134  176.869995  178.449997  176.270004  176.580002   

Price                   volume  
date       ticker               
2015-09-29 A         2252400.0  
           AAL       7478800.0  
           AAPL    293461600.0  
           ABBV     12842800.0  
           ABT      12287500.0  
...                        ...  
2023-09-26 XYL       1322400.0  
           YUM       1500600.0  
           ZBH       3610500.0  
           ZBRA       355400.0  
           ZTS       1463200.0  

[985955 rows x 6 columns]

<h3>2. Calculate features and technical indicators for each stock. </h3>
<ul>
    <li>Garman-Klass Volatility</li>
    <li>RSI</li>
    <li>Bollinger Bands</li>
    <li>ATR</li>
    <li>MACD</li>
    <li>Dollar Volume</li>
</ul>

# Garman-Klass Volatility Calculation

The Garman-Klass Volatility formula is given by:

$$
\sigma_{GK}^2 = \frac{1}{2} \ln^2 \left(\frac{High}{Low}\right) - \left(2 \ln(2) - 1\right) \ln^2 \left(\frac{Close}{Open}\right)
$$

This formula takes into account the high, low, close, and open prices of a stock.


In [32]:
# df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

# df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))
# df.xs('AAPL', level=1)['rsi'].plot()

# pandas_ta.bbands(close=df.xs('AAPL', level=1)['adj close'], length=20)
# df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
# df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
# df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])
# df

#Calculate ATR Normalized for Machine Learning Model and Cluster the data
# def compute_atr(stock_data):
#     atr = pandas_ta.atr(high=stock_data['high'],
#                         low=stock_data['low'],
#                         close=stock_data['close'],
#                         length=14)
#     return atr.sub(atr.mean()).div(atr.std())
# df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)
# df

#Calculate MACD Normalized 
# def compute_macd(close):
#     macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
#     return macd.sub(macd.mean()).div(macd.std())
# df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)
# df

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.425238   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.651133   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.004166   52.790001   54.189999   51.880001   53.099998   
           ABT      33.302025   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.736298   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211006  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.131134  176.869995  178.449997  176.270004  176.580002   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-09-29 A         2252400.0         -0.001082        NaN       NaN   
           AAL       7478800.0         -0.000443        NaN       NaN   
           AAPL    293461600.0         -0.005712        NaN       NaN   
           ABBV     12842800.0         -0.057368        NaN       NaN   
           ABT      12287500.0         -0.010064        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 XYL       1322400.0         -0.000018  26.146750  4.485761   
           YUM       1500600.0         -0.000051  36.057162  4.811707   
           ZBH       3610500.0          0.000022  31.893252  4.745884   
           ZBRA       355400.0          0.000133  29.494977  5.400991   
           ZTS       1463200.0          0.000049  42.623474  5.153746   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2015-09-29 A            NaN       NaN       NaN       NaN      70.782205  
           AAL          NaN       NaN       NaN       NaN     279.420098  
           AAPL         NaN       NaN       NaN       NaN    7234.160810  
           ABBV         NaN       NaN       NaN       NaN     462.394299  
           ABT          NaN       NaN       NaN       NaN     409.198630  
...                     ...       ...       ...       ...            ...  
2023-09-26 XYL     4.567684  4.649607  0.033800 -2.159189     117.344880  
           YUM     4.841672  4.871637  0.142547 -1.363696     183.389836  
           ZBH     4.785551  4.825217 -0.381708 -0.881067     402.696470  
           ZBRA    5.539167  5.677342 -0.057389 -1.600791      79.595386  
           ZTS     5.212559  5.271371  0.651515 -1.188278     256.251875  

[985955 rows x 14 columns]

<h3>3.Aggregate to monthly level and filter top 150 most liquid stocks for each month.</h3>
<ul>
    <li>To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.</li>
</ul>

In [45]:
#Dollar Volume aggregation
# last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
#                                                          'high','low','close']]
# last_cols
# df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume')

#Stack into multi index
data = pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
          df.unstack()[last_cols].resample('M').last().stack('ticker') ], axis=1).dropna()
data

dollar_volume   adj close  garman_klass_vol        rsi  \
date       ticker                                                           
2015-11-30 A          135.740917   38.950893         -0.002098  73.421326   
           AAL        287.915786   39.429928         -0.000966  40.718966   
           AAPL      4023.983939   26.854128         -0.003307  55.537275   
           ABBV       334.491186   40.025845         -0.062372  49.376878   
           ABT        210.542063   38.091496         -0.011927  56.962648   
...                          ...         ...               ...        ...   
2023-09-30 OTIS       154.361752   78.356506         -0.000097  33.116259   
           ABNB      1633.500725  132.279999          0.000213  44.494127   
           CEG        196.670368  107.862030          0.000131  55.245482   
           GEHC       212.197215   66.105721          0.000185  40.922336   
           KVUE       670.804320   20.006195         -0.000159  35.706316   

                     bb_low    bb_mid   bb_high       atr      macd  
date       ticker                                                    
2015-11-30 A       3.544191  3.616636  3.689080 -1.033887  0.567158  
           AAL     3.672028  3.749832  3.827635  0.190822 -0.418772  
           AAPL    3.281679  3.324991  3.368302 -0.967900 -0.142790  
           ABBV    3.717771  3.766009  3.814247 -0.526809  0.145677  
           ABT     3.650901  3.672752  3.694603 -1.064842  0.335558  
...                     ...       ...       ...       ...       ...  
2023-09-30 OTIS    4.370137  4.415425  4.460712 -1.028320 -1.534535  
           ABNB    4.857047  4.940924  5.024801 -1.006939 -0.037854  
           CEG     4.652147  4.692320  4.732493 -0.436215  0.366876  
           GEHC    4.155071  4.212607  4.270142 -0.893478 -1.116463  
           KVUE    3.014275  3.089221  3.164168 -0.899746 -1.435621  

[46553 rows x 9 columns]

<ul>
    <li>Calculate 5-year rolling average of dollar volume for each stocks before filtering.</li>
</ul>

In [58]:
#Rolling Average Dollar Volume for each stock
# data['dollar_volume']=(data['dollar_volume'].unstack('ticker').rolling(5*12).mean().stack())
# data

#Calculate Dollar Volume Rank Cross Sectionally for each month
# data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))
# data.dropna()

#Get top 150 most liquid stocks for each month
data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)
data

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2020-10-31 AAL      11.280000          0.000622  41.027057  2.483765   
           AAPL    106.506012          0.000078  43.770324  4.686059   
           ABBV     73.054321         -0.005843  47.517616  4.256773   
           ABT      98.313576         -0.001289  48.047136  4.595915   
           ACN     205.613708         -0.000628  42.280817  5.318201   
...                       ...               ...        ...       ...   
2023-09-30 VRTX    351.690002          0.000037  52.406728  5.838959   
           VZ       30.755373         -0.001680  42.222483  3.451862   
           WFC      39.781090         -0.000110  40.920295  3.697054   
           WMT      53.597542         -0.000041  54.722568  3.984976   
           XOM     113.372101         -0.000065  59.440183  4.687091   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2020-10-31 AAL     2.592127  2.700490 -1.244464 -0.348861  
           AAPL    4.744877  4.803695  1.409489 -0.628203  
           ABBV    4.300160  4.343547 -0.194581 -0.994069  
           ABT     4.623247  4.650579  0.983090 -0.069481  
           ACN     5.367234  5.416267  0.515540 -0.949113  
...                     ...       ...       ...       ...  
2023-09-30 VRTX    5.859127  5.879295  0.029799  0.027907  
           VZ      3.484054  3.516245 -1.078816 -0.350386  
           WFC     3.737418  3.777782 -0.558742 -0.282325  
           WMT     4.002446  4.019915 -0.196381  0.399459  
           XOM     4.727187  4.767283  0.601335  1.400623  

[5364 rows x 8 columns]